In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import display

# Метод Рунге–Кутты 2-го порядка
def rk2_step(f, s, y, h):
    k1 = f(s, y)
    k2 = f(s + h, y + h*k1)
    return y + 0.5*h*(k1 + k2)

def system_rhs(t, y):
    y1, y2 = y
    return np.array([
        y1 / (2 + 2*t) - 2*t*y2,
        y2 / (2 + 2*t) + 2*t*y1
    ])
def exact_solution(t):
    return np.array([
        np.cos(t**2) * np.sqrt(1 + t),
        np.sin(t**2) * np.sqrt(1 + t)
    ])

h = 0.01
t_range = np.arange(0, 2, h)
y = np.array([1.0, 0.0])

y1_sol = []
y2_sol = []
y1_exact = []
y2_exact = []

# Интеграция методом Рунге-Кутты
for t in t_range:
    y1_sol.append(y[0])
    y2_sol.append(y[1])
    y1_exact.append(exact_solution(t)[0])
    y2_exact.append(exact_solution(t)[1])
    y = rk2_step(system_rhs, t, y, h)


fig = go.Figure()
fig.add_trace(go.Scatter(x=t_range, y=y1_sol, mode='lines', name='y1 RK2'))
fig.add_trace(go.Scatter(x=t_range, y=y1_exact, mode='lines', name='y1 Exact', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=t_range, y=y2_sol, mode='lines', name='y2 RK2'))
fig.add_trace(go.Scatter(x=t_range, y=y2_exact, mode='lines', name='y2 Exact', line=dict(dash='dash')))
fig.update_layout(
    title="Решение системы уравнений",
    xaxis_title="t",
    yaxis_title="y"
)
display(fig)

hs = [0.1, 0.05, 0.025, 0.0125, 0.00625]
errs = []

for h in hs:
    t_range = np.arange(0, 2, h)
    y = np.array([1.0, 0.0])

    for t in t_range:
        y = rk2_step(system_rhs, t, y, h)
    y_exact = exact_solution(2.0)
    errs.append(np.max(np.abs(y - y_exact)))

# График e(h)
fig_e = go.Figure(go.Scatter(x=hs, y=errs, mode='lines+markers'))
fig_e.update_layout(
    title="Ошибка e(h) для системы уравнений",
    xaxis=dict(type='log', title='h'),
    yaxis=dict(type='log', title='e(h)')
)
display(fig_e)

# График e(h)/h^2
norm = [e/h**2 for e, h in zip(errs, hs)]
fig_n = go.Figure(go.Scatter(x=hs, y=norm, mode='lines+markers'))
fig_n.update_layout(
    title="Нормированная ошибка e(h)/h²",
    xaxis=dict(type='log', title='h'),
    yaxis=dict(title='e/h²')
)
display(fig_n)

# === Таблица экспериментального порядка α ===
alphas = [
    np.log(errs[i] / errs[i+1]) / np.log(hs[i] / hs[i+1])
    for i in range(len(hs)-1)
]

print(f"{'   h':>8s} {'   e(h)':>10s} {'  e/h²':>10s} {'   α':>6s}")
for i, h in enumerate(hs):
    a_str = f"{alphas[i]:.4f}" if i < len(alphas) else "  --  "
    print(f"{h:8.4f} {errs[i]:10.6e} {norm[i]:10.4e} {a_str:>6s}")
print(f"\nСредний α ≈ {np.mean(alphas):.4f}  (ожидается ≈2)\n")


def drop_rhs(s, y, alpha):
    phi, r, Z = y
    #ур-я 2 и 3
    return np.array([
        -np.sin(phi)/r - Z/alpha**2,
        np.cos(phi),
        -np.sin(phi)
    ])

h = float(input("Шаг h по дуге s: "))
alpha = float(input("Капиллярная постоянная alpha: "))
integr=1.5
z0_list = [0.3, 0.5, 0.7]  
phi0_list_deg = [30, 45, 60] 
phi0_list = [np.radians(x) for x in phi0_list_deg]

fig_drop = go.Figure()
fig_r = go.Figure()
fig_z = go.Figure()
fig_phi = go.Figure()

for z0 in z0_list:
    for phi0, phi0_deg in zip(phi0_list, phi0_list_deg):
        # Сдвиг старта, чтобы r!=0. замеч 2.1
        s = h
        #Начальные условия (4) со сдвигом
        y = np.array([
            phi0,
            h*np.cos(phi0),
            -z0 - h*np.sin(phi0)
        ])
        traj = []
        while y[0] <= integr:  #2.1
            traj.append([s, *y])
            y = rk2_step(lambda ss, yy: drop_rhs(ss, yy, alpha), s, y, h)
            s += h
        
        if not traj:
            print(f"Для z0={z0}, phi0={phi0_deg}° решение не найдено")
            continue
        
        traj = np.array(traj)
        label = f"z0={z0}, φ0={phi0_deg}°"
        r_values = traj[:, 1]
        z_values = traj[:, 2]
        reflected_r = 3.0 - r_values  
        full_r = np.concatenate([reflected_r[::-1], r_values])
        full_z = np.concatenate([z_values[::-1], z_values])
        # Добавляем профиль капли
        fig_drop.add_trace(go.Scatter(
            x=full_r, y=full_z,
            mode='lines',
            name=label,
        ))
        
        fig_r.add_trace(go.Scatter(
            x=traj[:, 0], y=traj[:, 1],
            mode='lines',
            name=label
        ))
        
        fig_z.add_trace(go.Scatter(
            x=traj[:, 0], y=traj[:, 2],
            mode='lines',
            name=label
        ))
        
        fig_phi.add_trace(go.Scatter(
            x=traj[:, 0], y=np.degrees(traj[:, 0]),
            mode='lines',
            name=label
        ))

fig_drop.update_layout(
    title=f"Профили капель (α={alpha}, h={h})",
    xaxis_title="r",
    yaxis_title="Z",
    yaxis=dict(scaleanchor="x", scaleratio=1),
    legend_title="Параметры"
)
display(fig_drop)

fig_r.update_layout(
    title="Зависимость r(s)",
    xaxis_title="s",
    yaxis_title="r",
    legend_title="Параметры"
)
display(fig_r)

fig_z.update_layout(
    title="Зависимость Z(s)",
    xaxis_title="s",
    yaxis_title="Z",
    legend_title="Параметры"
)
display(fig_z)

fig_phi.update_layout(
    title="Зависимость φ(s)",
    xaxis_title="s",
    yaxis_title="φ, градусы",
    legend_title="Параметры"
)
display(fig_phi)


       h       e(h)       e/h²      α
  0.1000 8.538731e-02 8.5387e+00 2.0897
  0.0500 2.005972e-02 8.0239e+00 2.0544
  0.0250 4.829482e-03 7.7272e+00 2.0306
  0.0125 1.182052e-03 7.5651e+00 2.0163
  0.0063 2.921938e-04 7.4802e+00   --  

Средний α ≈ 2.0477  (ожидается ≈2)

